In [1]:
from pathlib import Path

# 输入：可以是单个 epub 文件，也可以是目录
INPUT_PATH = Path(r"E:\\量子波动速读预处理\\input_epub")          # 例：Path("./book.epub") 或 Path("./epubs")
OUTPUT_DIR = Path(r"E:\\量子波动速读预处理\\output_markdown")         # 输出 md 的目录
RECURSIVE = False                      # 输入是目录时，是否递归子目录
OVERWRITE = True                       # md 已存在时是否覆盖

# pandoc 输出参数
TO_FORMAT = "gfm"                      # gfm / markdown
WRAP = "none"                          # none / preserve / auto
EXTRACT_MEDIA = True                   # 是否解包图片等资源
ADD_TOC = False                        # 是否加目录（有些书会显得冗余）
VERBOSE = True                         # 打印日志

In [5]:
# Cell 2: 核心函数 + 执行
import shutil
import subprocess
from pathlib import Path


def check_pandoc():
    pandoc = shutil.which("pandoc")
    if pandoc is None:
        raise EnvironmentError(
            "未找到 pandoc。请先安装并确保在 PATH 中可用。\n"
            "可在终端执行 `pandoc --version` 验证。"
        )
    return pandoc


def collect_epubs(input_path: Path, recursive: bool = False):
    if input_path.is_file():
        if input_path.suffix.lower() != ".epub":
            raise ValueError(f"输入文件不是 .epub: {input_path}")
        return [input_path]

    if input_path.is_dir():
        pattern = "**/*.epub" if recursive else "*.epub"
        files = sorted(input_path.glob(pattern))
        return files

    raise FileNotFoundError(f"输入路径不存在: {input_path}")


def build_cmd(epub_file: Path, md_file: Path, media_dir: Path):
    cmd = [
        "pandoc",
        str(epub_file),
        "-f", "epub",
        "-t", TO_FORMAT,
        f"--wrap={WRAP}",
        "-o", str(md_file),
    ]
    if EXTRACT_MEDIA:
        cmd.append(f"--extract-media={media_dir}")
    if ADD_TOC:
        cmd.append("--toc")
    return cmd


def convert_one(epub_file: Path, output_dir: Path, overwrite: bool = True, verbose: bool = True):
    output_dir.mkdir(parents=True, exist_ok=True)

    md_file = output_dir / f"{epub_file.stem}.md"
    media_dir = output_dir / f"{epub_file.stem}_assets"

    if md_file.exists() and not overwrite:
        if verbose:
            print(f"[SKIP] 已存在: {md_file}")
        return {"status": "skipped", "epub": epub_file, "md": md_file, "media": media_dir}

    cmd = build_cmd(epub_file, md_file, media_dir)

    try:
        result = subprocess.run(
            cmd,
            check=True,
            capture_output=True,
            text=True,
            encoding="utf-8",
            errors="replace",
        )
        if verbose:
            print(f"[OK] {epub_file.name} -> {md_file.name}")
            if result.stderr.strip():
                print("  pandoc stderr:", result.stderr.strip()[:500])
        return {"status": "ok", "epub": epub_file, "md": md_file, "media": media_dir}
    except subprocess.CalledProcessError as e:
        if verbose:
            print(f"[FAIL] {epub_file}")
            print("  returncode:", e.returncode)
            if e.stdout:
                print("  stdout:", e.stdout[:1000])
            if e.stderr:
                print("  stderr:", e.stderr[:1000])
        return {"status": "fail", "epub": epub_file, "md": md_file, "media": media_dir, "error": str(e)}


# ---- 执行入口 ----
check_pandoc()

epubs = collect_epubs(INPUT_PATH, recursive=RECURSIVE)
if not epubs:
    raise FileNotFoundError(f"未找到 epub 文件：{INPUT_PATH}")

print(f"待转换数量: {len(epubs)}")
results = []
for f in epubs:
    r = convert_one(f, OUTPUT_DIR, overwrite=OVERWRITE, verbose=VERBOSE)
    results.append(r)

ok = sum(1 for x in results if x["status"] == "ok")
sk = sum(1 for x in results if x["status"] == "skipped")
fl = sum(1 for x in results if x["status"] == "fail")
print(f"\n完成: ok={ok}, skipped={sk}, fail={fl}")


待转换数量: 4
[OK] 《大崛起》中国经济的增长与转型+-+赵燕菁.epub -> 《大崛起》中国经济的增长与转型+-+赵燕菁.md
[OK] 中国财政史十六讲基于财政政治学的历史重撰 (刘守刚) (Z-Library).epub -> 中国财政史十六讲基于财政政治学的历史重撰 (刘守刚) (Z-Library).md
[OK] 十六世纪明代中国之财政与税收.epub -> 十六世纪明代中国之财政与税收.md
[OK] 转型中的地方政府.epub -> 转型中的地方政府.md

完成: ok=4, skipped=0, fail=0


In [3]:
# Cell 3: （可选）结果表格查看
import pandas as pd

df = pd.DataFrame([
    {
        "status": r["status"],
        "epub": str(r["epub"]),
        "md": str(r["md"]),
        "media": str(r["media"]),
        "error": r.get("error", "")
    }
    for r in results
])

df


,status,epub,md,media,error
0,ok,E:\量子波动速读预处理\input_epub\转型中的地方政府.epub,E:\量子波动速读预处理\output_markdown\转型中的地方政府.md,E:\量子波动速读预处理\output_markdown\转型中的地方政府_assets,
